##目標:爬新的進來，舊的保留
(原本的重複讀取(舊的)會有問題 youtube_crawler.py)

###預計作法
* 找到爬進來的id位置
* 下if else指令，讓id重複的不讀取

``` python 
#run指令這樣下
    YT_CHANNEL_ID="iyenlung" MONGO_URI="mongodb://localhost:27017/dockerdata" python youtube_crawler.py
```

In [1]:
from pycrawlers.examples.youtube import crawlers #爬youtube的套件,py3要用需修改

In [2]:
import pandas as pd 

In [3]:
from pymongo import MongoClient#mongo用在python所需的套件

In [4]:
import os#有關系統指令進來

In [5]:
#人家輸入YT_CHANNEL_ID="iyenlung" MONGO_URI="mongodb://localhost:27017/dockerdata" python youtube_crawler.py
YT_CHANNEL_ID = os.environ.get("YT_CHANNEL_ID", "iyenlung")#"TWuseRGroup"這位置是什麼？台灣使用群？
MONGO_URI = os.environ.get("MONGO_URI","mongodb://localhost:27017/agilearning")#specify the host and port explicitly

DB_NAME = os.environ.get("DB_NAME","agilearning")
LEARNING_RESOURCE_COLLECTION_NAME = os.environ.get("LEARNING_RESOURCE_COLLECTION_NAME","learningResources")


In [6]:
print ("(database,collection)="+str((DB_NAME,LEARNING_RESOURCE_COLLECTION_NAME)))

(database,collection)=('agilearning', 'learningResources')


In [7]:
mongo_cli = MongoClient(MONGO_URI)#pymongo.mongo_client.MongoClient下的class
learning_resources_collection = mongo_cli[DB_NAME][LEARNING_RESOURCE_COLLECTION_NAME]


In [8]:
learning_resources_collection #擺放位置

Collection(Database(MongoClient('localhost', 27017), u'agilearning'), u'learningResources')

In [9]:
video_data = crawlers.crawl_channel_uploads(YT_CHANNEL_ID)#爬哪一個頻道

next_page_list =  ['https://gdata.youtube.com/feeds/users/iyenlung/uploads/']
next_page_list =  [u'https://gdata.youtube.com/feeds/users/iyenlung/uploads?alt=json&start-index=26&max-results=25']
next_page_list =  []


In [10]:
video_data#檢視一下爬下來什麼

[{'author': u'Yen-Lung Tsai',
  'categories': [{u'scheme': u'http://schemas.google.com/g/2005#kind',
    u'term': u'http://gdata.youtube.com/schemas/2007#video'},
   {u'label': u'Science & Technology',
    u'scheme': u'http://gdata.youtube.com/schemas/2007/categories.cat',
    u'term': u'Tech'}],
  'description': u'Python \u8b80\u5165\u5957\u4ef6\u7684\u65b9\u6cd5\u3002',
  'title': u'3-2 pylab inline \u505a\u4e86\u4ec0\u9ebc',
  'ytid': u'_oXvbxFP2oI'},
 {'author': u'Yen-Lung Tsai',
  'categories': [{u'scheme': u'http://schemas.google.com/g/2005#kind',
    u'term': u'http://gdata.youtube.com/schemas/2007#video'},
   {u'label': u'Science & Technology',
    u'scheme': u'http://gdata.youtube.com/schemas/2007/categories.cat',
    u'term': u'Tech'}],
  'description': u'\u958b\u59cb\u6578\u64da\u5206\u6790\u91cd\u8981\u7684 array \u8cc7\u6599\u578b\u614b\u3002',
  'title': u'3-1 Array \u771f\u7684\u70ab',
  'ytid': u'ayFu8KKJR2A'},
 {'author': u'Yen-Lung Tsai',
  'categories': [{u'scheme': 

###把其中一個拿出來看
```python
{'author': u'TW use-R',
 'categories': [{u'scheme': u'http://schemas.google.com/g/2005#kind',
                 u'term': u'http://gdata.youtube.com/schemas/2007#video'},
                {u'label': u'Science & Technology',
                 u'scheme': u'http://gdata.youtube.com/schemas/2007/categories.cat',
                 u'term': u'Tech'}],
 'description': u'\u8b1b\u984c\uff1aTITLE\n\u8b1b\u8005\uff1aSPEAKER\n\nAbstract\n\n[Abstract or Description]\n\nAbout                 the Speaker\n\n[About the Speaker]',
 'title': u'20150302 MLDM Monday x \u85cf\u982d\u8a69\u7522\u751f\u5668 part4',
 'ytid': u'V6r-Z3l4bc8'},
```

##用pandas來協助“存” video_data<--整個頻道內的所有影片
```
video_data_df = pd.DataFrame(video_data)

video_data_df = video_data_df[["title","description","ytid"]]
video_data_df.columns = [u'ytTitle', u'ytDescription', u'ytid']
```

###場外插撥：感人肺腑的一刻
```python
video_data_df.head()<----本來有出錯
```
不過，把種種更新後就ok了
```
conda update conda
conda update pandas
conda update ipython-notebook
```

In [11]:
video_data_df = pd.DataFrame(video_data)
video_data_df.head()

,author,categories,description,title,ytid
0,Yen-Lung Tsai,[{u'term': u'http://gdata.youtube.com/schemas/...,Python 讀入套件的方法。,3-2 pylab inline 做了什麼,_oXvbxFP2oI
1,Yen-Lung Tsai,[{u'term': u'http://gdata.youtube.com/schemas/...,開始數據分析重要的 array 資料型態。,3-1 Array 真的炫,ayFu8KKJR2A
2,Yen-Lung Tsai,[{u'term': u'http://gdata.youtube.com/schemas/...,我們介紹兩個概念:\n\n1. while 迴圈\n2. 讀入一個套件,2-5 While 和猜數字遊戲,7K3FR9ykha4
3,Yen-Lung Tsai,[{u'term': u'http://gdata.youtube.com/schemas/...,"程式中另一個重要概念, 條件判斷。我們學會了這個, 大概程式的概念就有了, 可以寫不少有趣的程式。",2-4 條件判斷,CweOErLahkE
4,Yen-Lung Tsai,[{u'term': u'http://gdata.youtube.com/schemas/...,"我們寫個「多變數」的函數, 在 IPython 中存檔。外加學會怎麼做更棒的顯示控制。",2-3 程式的存檔,D8qec8PVAFQ


In [12]:
video_data_df = video_data_df[["title","description","ytid"]] 

In [13]:
video_data_df.columns = [u'ytTitle', u'ytDescription', u'ytid']#換個名字，並且用unicode

In [14]:
video_data_df.head()

,ytTitle,ytDescription,ytid
0,3-2 pylab inline 做了什麼,Python 讀入套件的方法。,_oXvbxFP2oI
1,3-1 Array 真的炫,開始數據分析重要的 array 資料型態。,ayFu8KKJR2A
2,2-5 While 和猜數字遊戲,我們介紹兩個概念:\n\n1. while 迴圈\n2. 讀入一個套件,7K3FR9ykha4
3,2-4 條件判斷,"程式中另一個重要概念, 條件判斷。我們學會了這個, 大概程式的概念就有了, 可以寫不少有趣的程式。",CweOErLahkE
4,2-3 程式的存檔,"我們寫個「多變數」的函數, 在 IPython 中存檔。外加學會怎麼做更棒的顯示控制。",D8qec8PVAFQ


##繼續整理
```
video_data_df["youtubeVideoId"] = video_data_df["ytid"]
video_data_df = video_data_df.drop("ytid",1)
video_data_df["_id"] = video_data_df["youtubeVideoId"].map(lambda xx:"YTV_"+xx)
video_data_df["type"] = "youtube"

learning_resources_collection.insert(video_data_df.to_dict(orient="records"))
```


In [15]:
video_data_df["youtubeVideoId"] = video_data_df["ytid"]

In [16]:
video_data_df.head()

,ytTitle,ytDescription,ytid,youtubeVideoId
0,3-2 pylab inline 做了什麼,Python 讀入套件的方法。,_oXvbxFP2oI,_oXvbxFP2oI
1,3-1 Array 真的炫,開始數據分析重要的 array 資料型態。,ayFu8KKJR2A,ayFu8KKJR2A
2,2-5 While 和猜數字遊戲,我們介紹兩個概念:\n\n1. while 迴圈\n2. 讀入一個套件,7K3FR9ykha4,7K3FR9ykha4
3,2-4 條件判斷,"程式中另一個重要概念, 條件判斷。我們學會了這個, 大概程式的概念就有了, 可以寫不少有趣的程式。",CweOErLahkE,CweOErLahkE
4,2-3 程式的存檔,"我們寫個「多變數」的函數, 在 IPython 中存檔。外加學會怎麼做更棒的顯示控制。",D8qec8PVAFQ,D8qec8PVAFQ


In [17]:
video_data_df = video_data_df.drop("ytid",1)#0對index,1對column.把它弄掉用drop

In [18]:
video_data_df.head()

,ytTitle,ytDescription,youtubeVideoId
0,3-2 pylab inline 做了什麼,Python 讀入套件的方法。,_oXvbxFP2oI
1,3-1 Array 真的炫,開始數據分析重要的 array 資料型態。,ayFu8KKJR2A
2,2-5 While 和猜數字遊戲,我們介紹兩個概念:\n\n1. while 迴圈\n2. 讀入一個套件,7K3FR9ykha4
3,2-4 條件判斷,"程式中另一個重要概念, 條件判斷。我們學會了這個, 大概程式的概念就有了, 可以寫不少有趣的程式。",CweOErLahkE
4,2-3 程式的存檔,"我們寫個「多變數」的函數, 在 IPython 中存檔。外加學會怎麼做更棒的顯示控制。",D8qec8PVAFQ


In [19]:
video_data_df["_id"] = video_data_df["youtubeVideoId"].map(lambda xx:"YTV_"+xx)

In [20]:
video_data_df.head()

,ytTitle,ytDescription,youtubeVideoId,_id
0,3-2 pylab inline 做了什麼,Python 讀入套件的方法。,_oXvbxFP2oI,YTV__oXvbxFP2oI
1,3-1 Array 真的炫,開始數據分析重要的 array 資料型態。,ayFu8KKJR2A,YTV_ayFu8KKJR2A
2,2-5 While 和猜數字遊戲,我們介紹兩個概念:\n\n1. while 迴圈\n2. 讀入一個套件,7K3FR9ykha4,YTV_7K3FR9ykha4
3,2-4 條件判斷,"程式中另一個重要概念, 條件判斷。我們學會了這個, 大概程式的概念就有了, 可以寫不少有趣的程式。",CweOErLahkE,YTV_CweOErLahkE
4,2-3 程式的存檔,"我們寫個「多變數」的函數, 在 IPython 中存檔。外加學會怎麼做更棒的顯示控制。",D8qec8PVAFQ,YTV_D8qec8PVAFQ


In [21]:
video_data_df["type"] = "youtube" #可以一次把那一行通通變成同一文字
video_data_df.head()

,ytTitle,ytDescription,youtubeVideoId,_id,type
0,3-2 pylab inline 做了什麼,Python 讀入套件的方法。,_oXvbxFP2oI,YTV__oXvbxFP2oI,youtube
1,3-1 Array 真的炫,開始數據分析重要的 array 資料型態。,ayFu8KKJR2A,YTV_ayFu8KKJR2A,youtube
2,2-5 While 和猜數字遊戲,我們介紹兩個概念:\n\n1. while 迴圈\n2. 讀入一個套件,7K3FR9ykha4,YTV_7K3FR9ykha4,youtube
3,2-4 條件判斷,"程式中另一個重要概念, 條件判斷。我們學會了這個, 大概程式的概念就有了, 可以寫不少有趣的程式。",CweOErLahkE,YTV_CweOErLahkE,youtube
4,2-3 程式的存檔,"我們寫個「多變數」的函數, 在 IPython 中存檔。外加學會怎麼做更棒的顯示控制。",D8qec8PVAFQ,YTV_D8qec8PVAFQ,youtube


In [22]:
orient="records" #這個？

In [23]:
video_data_df.to_dict(orient="records")#video_data_df.to_dict(**kwargs)

[{'_id': u'YTV__oXvbxFP2oI',
  'type': 'youtube',
  'youtubeVideoId': u'_oXvbxFP2oI',
  u'ytDescription': u'Python \u8b80\u5165\u5957\u4ef6\u7684\u65b9\u6cd5\u3002',
  u'ytTitle': u'3-2 pylab inline \u505a\u4e86\u4ec0\u9ebc'},
 {'_id': u'YTV_ayFu8KKJR2A',
  'type': 'youtube',
  'youtubeVideoId': u'ayFu8KKJR2A',
  u'ytDescription': u'\u958b\u59cb\u6578\u64da\u5206\u6790\u91cd\u8981\u7684 array \u8cc7\u6599\u578b\u614b\u3002',
  u'ytTitle': u'3-1 Array \u771f\u7684\u70ab'},
 {'_id': u'YTV_7K3FR9ykha4',
  'type': 'youtube',
  'youtubeVideoId': u'7K3FR9ykha4',
  u'ytDescription': u'\u6211\u5011\u4ecb\u7d39\u5169\u500b\u6982\u5ff5:\n\n1. while \u8ff4\u5708\n2. \u8b80\u5165\u4e00\u500b\u5957\u4ef6',
  u'ytTitle': u'2-5 While \u548c\u731c\u6578\u5b57\u904a\u6232'},
 {'_id': u'YTV_CweOErLahkE',
  'type': 'youtube',
  'youtubeVideoId': u'CweOErLahkE',
  u'ytDescription': u'\u7a0b\u5f0f\u4e2d\u53e6\u4e00\u500b\u91cd\u8981\u6982\u5ff5, \u689d\u4ef6\u5224\u65b7\u3002\u6211\u5011\u5b78\u6703\u4e86\

##<b style="color:##FF2222"> 出現error!!!</b>
#開始嘗試

In [24]:
learning_resources_collection.insert(video_data_df.to_dict(orient="records"))#原文

DuplicateKeyError: E11000 duplicate key error index: agilearning.learningResources.$_id_ dup key: { : "YTV__oXvbxFP2oI" }

In [25]:
learning_resources_collection.insert(video_data_df).inserted_id#在第7行，把存的路徑弄好了

TypeError: 'unicode' object does not support item assignment

In [26]:
learning_resources_collection.insert(video_data_df.to_dict(orient="records")).inserted_ids #在第7行，把存的路徑弄好了

DuplicateKeyError: E11000 duplicate key error index: agilearning.learningResources.$_id_ dup key: { : "YTV__oXvbxFP2oI" }

In [27]:
learning_resources_collection.find()

In [28]:
learning_resources_collection.count()#目前數量

575

In [29]:
for post in learning_resources_collection.find():
    print post['_id']

YTV_ayFu8KKJR2A
YTV_7K3FR9ykha4
YTV_CweOErLahkE
YTV_D8qec8PVAFQ
YTV_3ltvKycVRFs
YTV_MAd9kiPJ4nA
YTV_620807cPZMY
YTV_lKp_rMBWbZU
YTV_2OdLclmO_0Q
YTV_v4sk2MSKK3M
YTV_K0PxXOk_deo
YTV_uSHtQUXlpk4
YTV_tbpqDQzX_gg
YTV_3hPo-Gk4IXQ
YTV_OHnrY3i6BM8
YTV_d0gb7448xIA
YTV_mKBcgmzjNiE
YTV_ZqU0KGxcNn0
YTV_xB7cC1MfFXE
YTV_7RTR8thNx18
YTV_WB8hg6UZeY0
YTV_WbCN3rKJCz4
YTV_GlK2wlW2ih0
YTV_Lksm2NceLz0
YTV_3MG9CHCCqDs
YTV_bNOYkAh5UXE
YTV_4b8oLwtyIX0
YTV_XAJBiEodiSQ
YTV_6FjpScjBxkg
YTV_8MKD-3b5koo
YTV_NqXyh1rOy-s
YTV_vriBje-D2yU
YTV_g_g4Gfc4dD0
YTV_N13zkgTCu0g
YTV_V6r-Z3l4bc8
YTV_1XEb2hMDFMs
YTV_CdMzHLrmpi8
YTV_qUEB1EVsDes
YTV_yCMFT8WAs68
YTV_yFHRogfk8iA
YTV_ka7Ttj8s0qQ
YTV_O5J-OSY0biY
YTV_oZC13D7dTCQ
YTV_mzq9b7K8ox8
YTV_lXfI7dB9hxo
YTV_3BzZUcqxLxA
YTV_ytNOa9y2wCY
YTV__lmZODWPwpE
YTV_Iu_kmUdbL68
YTV_2gViPZE81rE
YTV_5_c0NbbiqWo
YTV_WQPtQczOOTM
YTV__UmJAb0Sohs
YTV_OJ374zHypCI
YTV_fEx9CXCIWqI
YTV_UqQMGpfL0A0
YTV_UKoANiVkP-Y
YTV_J_c9Vkei1lM
YTV_lPmA4ju55W4
YTV_sz_v0nVUbkw
YTV_Lqgv44srexg
YTV_paSYy3e5d3o
YTV_6INs

In [30]:
for inner in video_data_df.to_dict(orient="records"):
    print inner['_id']

YTV__oXvbxFP2oI
YTV_ayFu8KKJR2A
YTV_7K3FR9ykha4
YTV_CweOErLahkE
YTV_D8qec8PVAFQ
YTV_3ltvKycVRFs
YTV_MAd9kiPJ4nA
YTV_620807cPZMY
YTV_lKp_rMBWbZU
YTV_2OdLclmO_0Q
YTV_v4sk2MSKK3M
YTV_K0PxXOk_deo
YTV_uSHtQUXlpk4
YTV_tbpqDQzX_gg
YTV_3hPo-Gk4IXQ
YTV_OHnrY3i6BM8
YTV_d0gb7448xIA
YTV_mKBcgmzjNiE
YTV_ZqU0KGxcNn0
YTV_xB7cC1MfFXE
YTV_7RTR8thNx18
YTV_WB8hg6UZeY0
YTV_WbCN3rKJCz4
YTV_GlK2wlW2ih0
YTV_Lksm2NceLz0
YTV_3MG9CHCCqDs
YTV_bNOYkAh5UXE
YTV_4b8oLwtyIX0
YTV_XAJBiEodiSQ
YTV_6FjpScjBxkg
YTV_8MKD-3b5koo


In [31]:
learning_resources_collection.find_one()#把第一個取出來

{u'_id': u'YTV_ayFu8KKJR2A',
 u'type': u'youtube',
 u'youtubeVideoId': u'ayFu8KKJR2A',
 u'ytDescription': u'\u958b\u59cb\u6578\u64da\u5206\u6790\u91cd\u8981\u7684 array \u8cc7\u6599\u578b\u614b\u3002',
 u'ytTitle': u'3-1 Array \u771f\u7684\u70ab'}

###把其中一個取出來
```python
{
u'youtubeVideoId': u'ayFu8KKJR2A',
u'_id': u'YTV_ayFu8KKJR2A', ##決定針對他
u'ytTitle': u'3-1 Array \u771f\u7684\u70ab', 
u'ytDescription': u'\u958b\u59cb\u6578\u64da\u5206\u6790\u91cd\u8981\u7684 array \u8cc7\u6599\u578b\u614b\u3002', u'type': u'youtube'
}
```

In [32]:
learning_resources_collection.find({u"_id":u'YTV_ayFu8KKJR2A'})

In [33]:
learning_resources_collection.find_one({u"_id":u'YTV_ayFu8KKJR2A'})

{u'_id': u'YTV_ayFu8KKJR2A',
 u'type': u'youtube',
 u'youtubeVideoId': u'ayFu8KKJR2A',
 u'ytDescription': u'\u958b\u59cb\u6578\u64da\u5206\u6790\u91cd\u8981\u7684 array \u8cc7\u6599\u578b\u614b\u3002',
 u'ytTitle': u'3-1 Array \u771f\u7684\u70ab'}

In [34]:
learning_resources_collection.find_one({u"_id":u"YTV_NqXyh1rOy-s" })['_id']

u'YTV_NqXyh1rOy-s'

In [35]:
len(video_data_df.to_dict(orient="records"))

31

In [36]:
#測答案的區塊
a=10
c=0
d=len(video_data_df.to_dict(orient="records"))
b=learning_resources_collection.count()
for inter in video_data_df.to_dict(orient="records"):
    a=c
    for post in learning_resources_collection.find():
        if inter['_id']==post['_id']:
               c+=1
                # print inter['_id']
    if a==c:
        learning_resources_collection.insert(inter)
    
d1=len(video_data_df.to_dict(orient="records"))
b1=learning_resources_collection.count()
print d,b,c,d1,b1

31 575 31 31 575


In [37]:
#傳答案的區塊
a=1
c=0
for inter in video_data_df.to_dict(orient="records"):
    a=c
    for post in learning_resources_collection.find():
        if inter['_id']==post['_id']:
               c+=1
    if a==c:
        learning_resources_collection.insert(inter)
    
